# Script to grab Trades and PL by market and store them

Process:

1. Cycle through each market from a start date
2. Grab any trades (if at all) for that particular time window
3. Store them and calculate P&L

 ## Imports

In [ ]:
import datetime as dt
import pickle
import pandas as pd
import plotnine as p9
import analysis.analysis_tools.traderanalysis as ta

In [ ]:
class Container:
    pass

In [ ]:
start_date = dt.datetime(2025, 11, 1, 0, 0)
end_date = dt.datetime(2026, 2, 11, 13, 30)

### Loading

In [ ]:
with open('pl_results.pkl', 'rb') as f:
    cc = pickle.load(f)

In [ ]:
proxy_wallet = '0x912a58103662ebe2e30328a305bc33131eca0f92'   # ratue

In [ ]:
date = cc.date
start_date = cc.start_date
end_date = dt.datetime(2026, 2, 11, 21, 0)
outcomes = cc.outcomes
trade_list = cc.trade_list

In [ ]:
while True:
    print(f"Fetching data for {date}")
    try:
        outcome, trades = ta.get_market_outcome_sync(proxy_wallet, 'btc', date)
        if outcome is not None:
            print(f".... found {outcome.num_trades_api} trades")
            outcomes.append(outcome)
            trade_list.append(trades)
        else:
            print(f".... found 0 trades")
    except Exception as e:
        print(f'Encountered exception e: {e}.  Continuing to next date')

    date = date + dt.timedelta(minutes=15)
    if date >= end_date:
        break


## PLotting Cumulative P&L

In [ ]:
(
    pd.DataFrame(outcomes)
    .assign(
        d=lambda x: pd.to_datetime(x['date'].dt.date)
    )
    .groupby('d')
    .agg({'pnl': 'sum'})
    .assign(cpl=lambda x: x['pnl'].cumsum())
    .reset_index()
    # .to_clipboard(excel=True)
    .sort_values('d')
    .pipe(p9.ggplot)
    + p9.aes(x='d', y='cpl')
    + p9.geom_line()
)

In [ ]:
(
    pd.DataFrame(outcomes)
    .assign(cpl=lambda x: x['pnl'].cumsum())
    .pipe(p9.ggplot)
    + p9.aes(x='date', y='cpl')
    + p9.geom_line()
)

### Storing

In [ ]:
c = Container()
c.outcomes = outcomes
c.trade_list = trade_list
c.date = date
c.start_date = start_date
c.end_date = end_date

In [ ]:
with open('ratue_pl_results.pkl', 'wb') as f:
    pickle.dump(c, f)